# Loading BART Data into Postgres

In [1]:
import psycopg2 as pg

In [2]:
# Connecting to Postgres
conn = pg.connect("dbname=bart_twitter user=brynstark")
cur = conn.cursor() # Allows us to execute Postgres operations

# Creating a table for all the BART data -- there will only be 12 rows
cur.execute('''
            CREATE TABLE bart_data (
                data json )''')

with open('bart_schedules_by_route.json') as f:
    routes_lst = f.read().split('\n')  # Splitting up JSON objects, separated by new-line
    
counter = 0   # Keep track of rows added
lost_rows = 0   # Keep track of rows (route info) lost
for routesched in routes_lst:
    try:
        cur.execute('''
                    INSERT INTO bart_data (data)
                    VALUES (%s)
                    ''', (routesched,))
        conn.commit()  #  Committing after each execution makes a difference
        counter += 1
    except:
        lost_rows += 1  # Keep track of tweets that couldn't be added
        continue

print('Route schedules added', counter)  # Prints the number of tweets added to DB from current JSON file
print('Route schedules lost', lost_rows)  # Prints tweets not added to DB
print()

# Closing out connections
conn.commit()   # ABC: Always Be Committing
cur.close()
conn.close()

Route schedules added 12
Route schedules lost 1



Perfect. The "lost" route schedule is simply the last line of the file, which is empty.

Checking the table:

In [3]:
conn = pg.connect("dbname=bart_twitter user=brynstark")
cur = conn.cursor()

In [4]:
cur.execute("SELECT * FROM bart_data LIMIT 1;")

In [5]:
for record in cur:
    print(record)

({'uri': 'http://api.bart.gov/api/sched.aspx?cmd=routesched&route=1', 'date': '7/13/2016', 'sched_num': '39', 'message': {'special_schedule': 'Elevator out of service at 12th Street Station 7/11-15.  <a href="http://www.bart.gov/news/articles/2016/news20160628">Read more...</a>'}, 'route': {'train': [{'@index': '1', 'stop': [{'@origTime': '4:02 AM', '@station': 'PITT', '@bikeflag': '1'}, {'@origTime': '4:08 AM', '@station': 'NCON', '@bikeflag': '1'}, {'@origTime': '4:12 AM', '@station': 'CONC', '@bikeflag': '1'}, {'@origTime': '4:17 AM', '@station': 'PHIL', '@bikeflag': '1'}, {'@origTime': '4:20 AM', '@station': 'WCRK', '@bikeflag': '1'}, {'@origTime': '4:25 AM', '@station': 'LAFY', '@bikeflag': '1'}, {'@origTime': '4:30 AM', '@station': 'ORIN', '@bikeflag': '1'}, {'@origTime': '4:35 AM', '@station': 'ROCK', '@bikeflag': '1'}, {'@origTime': '4:38 AM', '@station': 'MCAR', '@bikeflag': '1'}, {'@origTime': '4:42 AM', '@station': '19TH', '@bikeflag': '1'}, {'@origTime': '4:43 AM', '@statio

In [6]:
cur.close()
conn.close()

Sweet.